# Training

In [124]:
import pandas as pd
import numpy as np
X = pd.read_csv('data.csv')
y = pd.read_csv('marks.csv').values
y = np.reshape(y, (y.shape[0],))

In [125]:
keys = ['sent_num', 'index_text', 'index_sent', 'start_symb', 'end_symb']
X['Answer'] = y
for key in keys:
    X['diff_'+key] = X['Anaph:'+key]-X['Ant:'+key]
X['index'] = X['Anaph:'+keys[0]]
X['index'] = X['Anaph:index_text']
print(len(X['index'].unique()))
for key in keys:
    del X['Anaph:' + key], X['Ant:'+key]
y = X['Answer'].values
for i in [key for key in X if 'Lemma' in key]:
    print(i, len(X[(X['Answer'] == 1) & (X[i]==1)]))
del X['Answer']
for key in keys:
    if 'Token' in key:
        print(key)
X.head(10)

673
Anaph:TokenLemma:он 133
Anaph:TokenLemma:она 55
Anaph:TokenLemma:они 50
Anaph:TokenLemma:оно 3
Anaph:TokenLemma:такой 0
Anaph:TokenLemma:тот 8
Anaph:TokenLemma:этот 0


,Ant:ParentMorph:Animacy:Anim,Ant:ParentMorph:Animacy:Inan,Ant:ParentMorph:Aspect:Imp,Ant:ParentMorph:Aspect:Perf,Ant:ParentMorph:Case:Acc,Ant:ParentMorph:Case:Dat,Ant:ParentMorph:Case:Gen,Ant:ParentMorph:Case:Ins,Ant:ParentMorph:Case:Loc,Ant:ParentMorph:Case:Nom,...,Anaph:role:субъект_соучаст,Anaph:role:тематив,Anaph:role:темпоратив,Anaph:role:транзитив,diff_sent_num,diff_index_text,diff_index_sent,diff_start_symb,diff_end_symb,index
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,3,98,40,683,679,111
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,3,97,39,675,669,111
2,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,3,91,33,646,639,111
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,89,39,624,621,111
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,82,32,583,581,111
5,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,2,78,28,572,569,111
6,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,2,76,26,564,559,111
7,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,72,22,538,538,111
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,2,61,11,488,479,111
9,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,2,60,10,475,473,111


In [126]:
index, dist = X['index'].values, X['diff_' + 'index_sent'].values
res = {i:[] for i in index}
for ind, i in enumerate(index):
    res[i].append((dist[ind],ind))
def transform(res):
    list_ = []
    u = []
    for i in res:
        cur = res[i]
        cur.sort(key = lambda x: x[0])
        u.append(len(cur))
        list_ += [(ind, i[1]) for ind,i in enumerate(cur)]
    res = list_.copy()
    for i in list_:
        res[i[1]] = i[0]
    return res
X['Distance'] = transform(res)
del X['index']
keys = X.columns

In [127]:
import numpy as np
X = X.values
y = np.reshape(y, (y.shape[0],))

In [128]:
def my_train_test_split(X,y,index, size = 0.1):
    indexes = np.unique(index)
    size = int(len(indexes)*size)
    choice = np.random.choice(indexes, size)
    X_test, y_test, X_train, y_train = [], [], [], []
    index_test, index_train = [], []
    for ind,i in enumerate(index):
        if i in choice:
            X_test.append(X[ind])
            y_test.append(y[ind])
            index_test.append(i)
        else:
            X_train.append(X[ind])
            y_train.append(y[ind])
            index_train.append(i)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    index_train = np.array(index_train)
    index_test = np.array(index_test)
    return X_train, X_test, y_train, y_test, index_train, index_test

In [129]:
def metric(y_pred, y, index):
    res =dict()
    S = 0
    true = dict()
    for ind,i in enumerate(y):
        if i==1:
            true[index[ind]] = ind
            res[index[ind]] = []
            S += 1
    for ind, i in enumerate(y_pred):
        if index[ind] in res:
            res[index[ind]].append((ind,i))
    s = 0
    for i in res:
        s += int(max(res[i],key = lambda x: x[1])[0] == true[i])
    return s/S
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
def validate(X,y,clf, metric = log_loss, n_splits = 10):
    kf = KFold(n_splits = n_splits)
    s = 0
    for train_index, test_index in kf.split(X):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        s += metric(y_test, clf.fit(X_train, y_train).predict_proba(X_test))
    return s/n_splits

In [171]:
X_train, X_test, y_train, y_test, index_train, index_test = my_train_test_split(X,y, index, 0.2)
print('Test', len([i for i in y_test if i==1]))
print('Train', len([i for i in y_train if i==1]))

Test 44
Train 205


# Gradient Boosting on Trees

In [172]:
import lightgbm as lgb
param = {'num_leaves': 6,
         'objective': 'binary',
         'metric' : ''}
num_round = 1000
train_data = lgb.Dataset(X_train, label=y_train)
bst = lgb.train(param,train_data, num_round)

In [173]:
from sklearn.metrics import f1_score
y_pred = bst.predict(X_test)
index_test = [int(i) for i in index_test]
metric(y_pred, y_test, index_test)

0.38636363636363635

# Random Forest

In [174]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 10000).fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)[:, 1]
index_test = [int(i) for i in index_test]
metric(y_pred, y_test, index_test)

0.38636363636363635

# Neural Network